In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

import pickle as cPickle
import pandas as pd
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from keras.layers import LSTM, Dense, TimeDistributed, Bidirectional
import sklearn.metrics
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional
from keras import models

In [ ]:
data_teacher = "/content/drive/MyDrive/KD2_Predict_ArapUbi/Data/Data for teacher/"
path_model = "/content/drive/MyDrive/KD2_Predict_ArapUbi/Model/"
path_result = "/content/drive/MyDrive/KD2_Predict_ArapUbi/Result/"

In [ ]:
def twoTupleDic3():
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        for j in AA_list_sort:
          for jj in AA_list_sort:
             AA_dict[i+j+jj] = numm
             numm += 1
    return AA_dict
def twoTupleDic2():
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        for j in AA_list_sort:
          AA_dict[i+j] = numm
          numm += 1
    return AA_dict

def twoTupleDic1():
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        AA_dict[i] = numm
        numm += 1
    return AA_dict
def ProSentence(pro, K):
	sentence = ""
	length = len(pro)
	for i in range(length - K + 1):
		sentence += pro[i: i + K] + " "
    #delete extra space
	sentence = sentence[0 : len(sentence) - 1]
	return sentence
k =1#1-gram
word_index1 = twoTupleDic1()
vocab_size = len(word_index1)
num_folds = 5
TIME_STEPS = 33
INPUT_SIZE = 300

**Load data for teacher training**

In [ ]:
# Load Traindata
file_train_teacher = "train_data_3speaces_31.csv"
df_train_teacher =pd.read_csv(data_teacher +file_train_teacher, delimiter= ',')
texts_train_teacher =[] #PTMsequend kmer
for i in df_train_teacher['Sequence']:
  temp = ProSentence(i,k) # Biểu diễn dữ liệu đầu vào thành token Kmer
  texts_train_teacher.append(temp)
df_train_teacher['k_mer'] =texts_train_teacher
train_sequences_teacher = []
for each in texts_train_teacher:
    each_index_list = []
    each = each.split(' ')
    for i in each:
        each_index_list.append(word_index1[i])
    train_sequences_teacher.append(each_index_list)
# Tokenizer train data
data_token_teacher = []
for i in df_train_teacher['k_mer']:
   data_token_teacher.append(i.split())

# Len of the K_mer[1]

MAX_SEQUENCE_LENGTH = len(data_token_teacher[1])

Xtrain_teacher = pad_sequences(train_sequences_teacher, maxlen=MAX_SEQUENCE_LENGTH)
ytrain_teacher = np.array(df_train_teacher['Label'])
Xtrain_teacher.shape
ytrain_teacher = np.array(ytrain_teacher)

# perform one-hot encoding on the labels
lb = LabelBinarizer()
ytrain_teacher = lb.fit_transform(ytrain_teacher)
ytrain_teacher = to_categorical(ytrain_teacher)
ytrain_teacher.shape

(6746, 2)

In [ ]:
# load test data for teacher
file_test_teacher ="test_data_3speaces_31.csv"
df_test_teacher =pd.read_csv(data_teacher+file_test_teacher,delimiter= ',')
text_test_teacher =[] #PTMsequend kmer
for i in df_test_teacher['Sequence']:
  temp = ProSentence(i,k) # Biểu diễn dữ liệu đầu vào thành token Kmer
  text_test_teacher.append(temp)
df_test_teacher['k_mer'] =text_test_teacher

test_sequences_teacher = []
for each in text_test_teacher:
    each_index_list = []
    each = each.split(' ')
    for i in each:
        each_index_list.append(word_index1[i])
    test_sequences_teacher.append(each_index_list)

Xtest_teacher = pad_sequences(test_sequences_teacher, maxlen=MAX_SEQUENCE_LENGTH)
ytest_teacher = np.array(df_test_teacher['Label'])
Xtest_teacher.shape
ytest_teacher = np.array(ytest_teacher)
# perform one-hot encoding on the labels
lb = LabelBinarizer()
ytest_teacher= lb.fit_transform(ytest_teacher)
ytest_teacher = to_categorical(ytest_teacher)
ytest_teacher.shape

(1500, 2)

**Define Teacher model**

In [ ]:

teacher = keras.Sequential(
    [
        layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True),
        #layers.Embedding(len(embedding_matrix), EMBEDDING_DIM, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH_teacher,trainable=True),
        layers.Bidirectional(LSTM(units=32,batch_input_shape=(None,TIME_STEPS, INPUT_SIZE),return_sequences=True)),
        layers.Dropout(0.2),
        layers.Flatten(),
        layers.Dense(128),
        layers.Activation('relu'),
        layers.Dropout(0.2),
        layers.Dense(2),
        layers.Activation('softmax'),
    ],
    name="teacher",
)

teacher.compile(
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=[
        "categorical_accuracy",
        "AUC",

    ],
    optimizer=keras.optimizers.AdamW(learning_rate=0.0001),
)
checkpoint = EarlyStopping(monitor='val_loss',
            min_delta=0,
            patience=3,
            verbose=1, mode='auto')

Teacher train

In [ ]:
# Train teacher and save model
teacher.fit(Xtrain_teacher, ytrain_teacher, batch_size=16, epochs=100,validation_data =(Xtest_teacher, ytest_teacher), shuffle = True,callbacks=[checkpoint],verbose=1)
teacher.save(path_result +"teacher_KD2_1gram.h5")

Epoch 1/100
422/422 [==============================] - 8s 19ms/step - loss: 0.4076 - categorical_accuracy: 0.8217 - auc: 0.8973 - val_loss: 0.3843 - val_categorical_accuracy: 0.8400 - val_auc: 0.9101
Epoch 2/100
422/422 [==============================] - 8s 19ms/step - loss: 0.4011 - categorical_accuracy: 0.8257 - auc: 0.9002 - val_loss: 0.3770 - val_categorical_accuracy: 0.8420 - val_auc: 0.9131
Epoch 3/100
422/422 [==============================] - 7s 18ms/step - loss: 0.3947 - categorical_accuracy: 0.8318 - auc: 0.9034 - val_loss: 0.3869 - val_categorical_accuracy: 0.8427 - val_auc: 0.9085
Epoch 4/100
422/422 [==============================] - 8s 18ms/step - loss: 0.3861 - categorical_accuracy: 0.8352 - auc: 0.9082 - val_loss: 0.3774 - val_categorical_accuracy: 0.8413 - val_auc: 0.9128
Epoch 5/100
422/422 [==============================] - 7s 17ms/step - loss: 0.3815 - categorical_accuracy: 0.8375 - auc: 0.9104 - val_loss: 0.3815 - val_categorical_accuracy: 0.8513 - val_auc: 0.9112
